In [1]:
from mako.template import Template
import pandas as pd


In [4]:
import requests,os
from bs4 import BeautifulSoup

os.environ['https_proxy'] = '127.0.0.1:7890'
os.environ['http_proxy'] = '127.0.0.1:7890'

headers = {
    'Host': 'blog.udn.com',
    'Referer': 'https://blog.udn.com/MengyuanWang/article',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35'
}


In [15]:
list(set(['179164306', '129879105', '129017969', '114378430', '178109282'] + ['179164306', '129879105', '129017969', '114378430', '178109282']))

['114378430', '129879105', '129017969', '178109282', '179164306']

In [1]:
import requests,json
import pandas as pd

In [79]:
df0 = pd.read_excel('../data/table.xlsx')
df0 = df0.set_index(df0.columns[0],drop=True)
df0.to_csv('table.csv')

In [17]:
import requests,json
from bs4 import BeautifulSoup